***An Object-oriented Python optimizer used for linac optimization together with ASTRA and IMPACT-T***

----
# Definition of the optimization problem

\begin{equation}
Min\>f(x),\>w.r.t.\>x\>
\end{equation}
\begin{equation}
s.t.\>g_i(x)<=0,\>i = 1, ..., m
\end{equation}
\begin{equation}
x_{j,lower} <= x_j <= x_{j, upper},\>j=1,2,...,n
\end{equation}

where:
- $f(x)$ is a nonlinear function;
- $g_i(x)$ is a group of linear or nonlinear functions;
- $x_j$ is the vector of design variables;
- $m$ is the number of constraints.
- $n$ is the number of design variables;

----
# System requirements:

- Linux (Ubuntu 14.04, 16.04)
- Python2 > 2.7.6
- [NumPy](http://www.numpy.org/), [Pandas](http://pandas.pydata.org/), [Scipy](https://www.scipy.org/scipylib/index.html)
- [matplotlib](http://matplotlib.org/)(only for visualization)
- [pyOpt](http://www.pyopt.org/)
- [ASTRA](http://www.desy.de/~mpyflo/)
- [IMPACT-T](http://portal.nersc.gov/project/m669/IMPACT-T/)

----
# Modified pyOpt (not mandatory)
**alpso.py** - Location in the original package: _pyopt/pyOpt/pyALPSO/alpso.py_

The new file will let you know why the optimization converges and print out the value of the augmented Lagrangian term.

----
# Examples

  There are two examples (astra_test.py and impact_test.py) in the folder.

----
# Setup and run an optimization problem

## Step 1: Assign system path and import modules

In [3]:
import sys
import os
sys.path.append(os.path.expanduser('~') + "/myscripts/linac_optimization/")

from linac_opt import LinacOpt

## Step 2: Instantiate the optimization

In [4]:
opt_test = LinacOpt(path_name='./astra_test',
                    input_file='injector.in',
                    input_template='injector.in.000',
                    particle_type='astra',
                    prob_name='opt_test',
                    restart=None,
                    run_once=False)

## Step 3: Set up the optimizer

Global optimizers: [ALPSO](http://www.pyopt.org/reference/optimizers.alpso.html#module-pyALPSO) and [NSGA2](http://www.pyopt.org/reference/optimizers.nsga2.html#module-pyNSGA2)

Local search optimizers: [SDPEN](http://www.pyopt.org/reference/optimizers.sdpen.html#module-pySDPEN)

_Tips for ALPSO optimizer:_
- Convergence condition:
```
    if (abs(global_distance[0]-global_distance[stopIters-1]) <= \
      dtol*abs(global_distance[stopIters-1]) and \
      abs(global_L[0]-global_L[stopIters-1]) <= \
      rtol*abs(global_L[stopIters-1]) or \
      abs(global_L[0]-global_L[stopIters-1]) <= atol):
      stop_criteria_flag = 1
```
    If the objective is a small number, one must change 'atol' otherwise the optimization will stop prematurely.

- 'dynInnerIter' must be set to 1 in order to make minInnerIter take effect!

In [5]:
opt_test.set_optimizer('alpso')

opt_test.optimizer.setOption('atol', 1e-2)
opt_test.optimizer.setOption('rtol', 1e-2)
opt_test.optimizer.setOption('SwarmSize', 10)  # For speed
opt_test.optimizer.setOption('maxInnerIter', 2)  # For speed
opt_test.optimizer.setOption('minInnerIter', 1)
opt_test.optimizer.setOption('dynInnerIter', 1)
opt_test.optimizer.setOption('stopIters', 2)
opt_test.optimizer.setOption('w1', 0.99)
opt_test.optimizer.setOption('w2', 0.40)
opt_test.optimizer.setOption('c1', 2.0)
opt_test.optimizer.setOption('c2', 2.0)

## Step 4: Add fit-points and sections

In [4]:
opt_test.fit_points.set_point('out', 'injector.0600.001', cut_tail=0.1)
opt_test.fit_points.set_point('slit', 'injector.0029.001', cut_halo=0.1)

In [5]:
opt_test.sections.set_section('all')
opt_test.sections.set_section('tws2', z_lim=(3.7, 5.7))

## Step 5: Set objective and constraints

In [6]:
def f1(fits):
    return fits.out.St*1.e15
opt_test.opt_prob.set_obj('St_fs', f1)

In [7]:
def g1(fits):
    return fits.out.n0
opt_test.opt_prob.set_con('n_pars', g1, equal=500, tol=0.0)

def g2(fits):
    return fits.slit.Sx*1.e3
opt_test.opt_prob.set_con('Sx_at_slit_mm', g2, upper=1.3)

def g3(fits, sections):
    return sections.all.Sx.max*1.e3
opt_test.opt_prob.set_con('max_Sx_mm', g3, upper=10.)

def g4(fits, sections):
    return sections.tws2.emitx.max*1.e6
opt_test.opt_prob.set_con('max_emitx_TWS2_um', g4, upper=0.3)

## Step 6: Add variables, co-variables and static-variables


In [8]:
opt_test.opt_prob.set_var('laser_spot', value=0.2, lower=0.1, upper=0.5)
opt_test.opt_prob.set_var('hc_sole_b', value=0.0, lower=0.0, upper=1.0)
opt_test.opt_prob.set_var('main_sole_b', value=0.0, lower=0.0, upper=0.4)
opt_test.opt_prob.set_var('tws1_sole_b', value=0.0, lower=0.0, upper=0.1)
opt_test.opt_prob.set_var('tws1_phase', value=0.0, lower=-90.0, upper=0.0)
opt_test.opt_prob.set_var('tws2_phase', value=0.0, lower=-90.0, upper=0.0)

opt_test.opt_prob.set_covar('tws2_sole_b', 'tws1_sole_b', slope=1.0, intercept=0.0)

opt_test.opt_prob.set_staticvar('hmax', value=0.02)
opt_test.opt_prob.set_staticvar('hmin', value=0.001)
opt_test.opt_prob.set_staticvar('nrad', value=16)
opt_test.opt_prob.set_staticvar('nlong', value=16)

## Step 7: Run the optimization


In [9]:
run_code = 'mpirun -np 2 astra_r62_Linux_x86_64_OpenMPI_1.6.1'

opt_test.solve(run_code)

File removed: opt_test.log.001
File removed: opt_test.log.000
File removed: opt_test.sol.000
File removed: opt_test.sol.000.pkl

********************************************************************************
Start solving the following problem with pyOpt.ALPSO on 
mpirun -np 2 astra_r62_Linux_x86_64_OpenMPI_1.6.1 /home/jun/myscripts/linac_optimization/astra_test/injector.in
********************************************************************************

Optimization Problem -- opt_test-2017-01-24-09-24-22

Objectives:
  Name                Value        Optimum      Function        
  St_fs                0.0000e+00  -1.0000e+21  f1              

Constraints:
  Name                Value        Bound                                           
  n_pars               0.0000e+00  5.0000e+02 - 0.0000e+00 <= g1() <= 5.0000e+02 + 0.0000e+00
  Sx_at_slit_mm        0.0000e+00  -1.0000e+21 <= g2() <= 1.3000e+00               
  max_Sx_mm            0.0000e+00  -1.0000e+21 <= g3() <= 1.0000e+0

## Step 8: Run the refinement


In [10]:
# Change optimizer
opt_test.set_optimizer('sdpen')
opt_test.optimizer.setOption('alfa_stop', 1e-3)
opt_test.optimizer.setOption('iprint', 0)
opt_test.optimizer.setOption('nf_max', 200)

def f2(fits):
    """Add objective"""
    return (fits.out.emitx + fits.out.emitx)*1.e6/2
opt_test.opt_prob.set_obj('emit_um', f2)
opt_test.opt_prob.del_obj('St_fs')

opt_test.opt_prob.set_staticvar('hmax', value=0.01)
opt_test.opt_prob.set_staticvar('hmin', value=0.0005)
opt_test.opt_prob.set_staticvar('nrad', value=32)
opt_test.opt_prob.set_staticvar('nlong', value=32)

opt_test.opt_prob.set_staticvar('tws1_phase')
opt_test.opt_prob.set_staticvar('tws2_phase')

opt_test.solve(run_code)

tws1_phase was changed from variable to static variable!

tws2_phase was changed from variable to static variable!


********************************************************************************
Start solving the following problem with pyOpt.SDPEN on 
mpirun -np 2 astra_r62_Linux_x86_64_OpenMPI_1.6.1 /home/jun/myscripts/linac_optimization/astra_test/injector.in
********************************************************************************

Optimization Problem -- opt_test-2017-01-24-09-24-22

Objectives:
  Name                Value        Optimum      Function        
  emit_um              0.0000e+00  -1.0000e+21  f2              

Constraints:
  Name                Value        Bound                                           
  n_pars               5.0000e+02  5.0000e+02 - 0.0000e+00 <= g1() <= 5.0000e+02 + 0.0000e+00
  Sx_at_slit_mm        2.5898e-01  -1.0000e+21 <= g2() <= 1.3000e+00               
  max_Sx_mm            3.9688e-01  -1.0000e+21 <= g3() <= 1.0000e+01           

----
# Visualization

----
# Class definition

```
class LinacOpt(LinacOptData):
    def __init__(self, path_name=None, input_file=None, input_template=None,
                 particle_type=None, prob_name='opt_prob',
                 restart=None, run_once=False, max_fail=10):
        """Initialization.

        Parameters
        ----------
        path_name: string
            Path of the simulation directory.
        input_file: string
            Name of the input file.
        input_template: string
            Name of the input template file. The patterns (<string>) in
            the template file will be replaced at the beginning of
            each simulation to generate a new input file.
        particle_type: string
            Type of the particle file (corresponding code name).
        prob_name: string
            Root name of log_file and solution_file, default = 'opt_prob'.
        restart: None/int
            None for new run and int for restarting from the
            restart-th run.
        run_once: Boolean
            True for stopping after one simulation. For debug.
        max_fail: int
            Max number of allowed consecutive fails.
        """
```

```
class FitPoints(object):
    """Object with attributes being PhaseSpace objects"""
    def __init__(self, particle_type):
        self.particle_type = particle_type
        
    def set_point(self, name, particle_file, particle_type=None, **kwargs):
        """Add a PhaseSpace object as an attribute

        Parameters
        ----------
        name: string
            Name of the new attribute.
        particle_file: string
            Name of the particle file.
        particle_type: string
            Type of the particle file.
            
        Additional keyword arguments are passed to PhaseSpace.__init__().
        """

    def del_point(self, name):
        """Delete a point by name.

        Parameters
        ----------
        name: string
            Name of the new attribute.
        """


class PhaseSpace(LinacOptData):
    """Store the particle phase-space data and parameters
    
    Attributes
    ----------
    data: pandas.DataFrame object
        Columns: x (m), y (m), z (m), px, py, t (s, dt), p (mc)

    slice_percent: float
        Percentage of particle for slice parameters calculation.
    cut_halo: None/float
        Percentage of halo to be cut.
    cut_tail: None/float
        Percentage of tail to be cut.

    n0: int
        Number of particles in the original file.
    n: int
        Number of particles.
    p: float
        Average normalized momentum.
    gamma: float
        Average Lorentz factor.
    chirp: float
        Energy chirp (1/m), defined as -<z*delta>/<z^2>.
    Sdelta: float
        Fractional energy spread.
    Sdelta_slice: float
        Fractional slice energy spread.
    St: float
        RMS bunch duration (s).
    St_slice: float
        RMS slice bunch duration (s).
    Sz: float
        RMS bunch length (m).
    I_peak: float
        Peak current (A).
    Sdelta_un: float
        Uncorrelated momentum spread.
    emitx/emity: float
        Normalized horizontal/vertical emittance (m.rad).
    emitx_slice/emity_slice: float
        Normalized horizontal/vertical slice emittance (m.rad).
    Sx/Sy: float
        RMS horizontal/vertical beam size (m).
    betax/betay: float
        Horizontal/vertical beta function (m).
    alphax/alphay: float
        Horizontal/vertical alpha function (m).
    emitx_tr/emity_tr: float
        Normalized horizontal/vertical trace space emittance (m.rad)
    Cx/Cy: float
        Horizontal/vertical displacement (m).
    Cxp/Cyp: float
        Horizontal/vertical divergence (rad).
    Ct: float
        Average timing (s).
        
    def __init__(self, particle_file, particle_type, charge=0.0, q_norm=None,
                 slice_percent=0.1, cut_halo=None, cut_tail=None, opt=False):
        """
        Parameters
        ----------
        particle_file: string
            Name of the particle file.
        particle_type: string
            Type of the particle file.
        charge: int/float
            Charge of the beam (in C). Only for Impact data.
            Ignored if q_norm is given.
        q_norm: int/float
            Charge per macro-particle.
        slice_percent: float
            Percentage of particles for slice properties.
        cut_halo: None/float
            Percentage of particles to be removed based on their
            transverse distance to the bunch centroid.
        cut_tail: None/float
            Percentage of particles to be removed based on their
            longitudinal distance to the bunch centroid.
        opt: Boolean
            True for the initialization of the fit-points in linac_opt.
            Since there is no output, an error will occur if the update
            method is called. Default is False.
        """
        
    def update(self, current_bins='auto', filter_sigma=1):
        """Read the phase-space and calculate the beam parameters

        Parameters
        ----------
        current_bins: int/'auto'
            No. of bins to calculate the current profile. Only affect
            the calculation of the peak current.
        filter_sigma: int/float
            Standard deviation of the Gaussian kernel of the 1D Gaussian filter.
        """
```

```
class BeamEvolution(LinacOptData):
    """Store the beam evolution and its statistics

    Attributes
    ----------
    data: pandas.DataFrame object
        z (m), gamma, SdE (eV), Sx (m), Sy (m), Sz (m),
        emitx (m.rad), emity (m.rad), emitz (m.rad),
        emitx_tr (m.rad), emity_tr (m.rad),
        betax (m), betay (m), alphax, alphay

    z: Stats object
        Longitudinal position (m).
    gamma: Stats object
        Lorentz factor.
    SdE: Stats object
        RMS energy spread (eV).
    Sx/Sy/Sz: Stats objects
        RMS bunch sizes (m).
    betax/betay: Stats objects
        Beta functions (m).
    alphax/alphay: Stats objects
        Alpha functions.
    emitx/emity: Stats objects
        Normalized canonical emittance (m.rad).
    emitx_tr/emity_tr: Stats objects
        Normalized trace-space emittance (m.rad).
    """
    def __init__(self, root_name, particle_type, z_lim=None, opt=False):
        """Initialize BeamStats object

        Parameters
        ----------
        root_name: string
            The root name of the output files. For Impact-T files,
            root_name will be set to 'fort' if not given.
        particle_type: string
            Type of the particle file.
        z_lim: scalar/tuple
            If None, passed as (-INF, INF)
            if scalar, being passed as (left, INF)
            if tuple, the first two elements being passed as (left, right)
        opt: Boolean
            True for the initialization of the fit-points in linac_opt.
            Since there is no output, an error will occur if the update
            method is called. Default is False.
        """


class Stats(object):
    """Store the statistic values of an array-like object.

    Attributes
    ----------
    start: float
        First value.
    end: float
        Last value.
    max: float
        Maximum value.
    min: float
        Minimum value.
    ave: float
        Average value.
    std: float
        Standard deviation.
    """


class Sections(object):
    """Object with attributes being BeamEvolution objects"""
    def set_section(self, name, root_name=None, particle_type=None, **kwargs):
        """Add a BeamEvolution object as an attribute

        Parameters
        ----------
        name: string
            Name of the new attribute.
        root_name: string
            The root name of the output files.
        particle_type: string
            Type of the particle file.
            
        Additional keyword arguments are passed to BeamEvolution.__init__().
        """
        try:
            self.__delattr__(name)
            print "\nWarning: section {} will be replaced!".format(name)
        except AttributeError:
            pass

        if root_name is None:
            root_name = self.root_name

        if particle_type is None:
            particle_type = self.particle_type

        this_section = BeamEvolution(root_name, particle_type, opt=True, **kwargs)
        super(Sections, self).__setattr__(name, this_section)

    def del_section(self, name):
        """Delete a section by name.

        Parameters
        ----------
        name: string
            Name of the new attribute.
        """
        super(Sections, self).__delattr__(name)
```

```
class Objective(pyOptObjective):
    """Inherited from pyOpt.Objective class.

    Each Objective object has a function attribute which calculates 
    its value.
    """

    def __init__(self, name, func=None, optimum=-1.0e21):
        """Initialize Objective object

        Parameters
        ----------
        func: function object
            Objective function.
        optimum: None/float
            The object of the objective.
        """
```

```
class Constraint(pyOptConstraint):
    """Inherited from pyOpt.Constraint class.

    One can specify one of the three constraint formats:
    [-INF, upper], [lower, INF] and [equal - tol, equal + tol]
    """
    def __init__(self, name, func=None, **kwargs):
        """Initialize Constraint object.

        Parameters
        ----------
        func: function object
            Name of the constraint function.
        upper: float
            Upper boundary, ignored if lower if specified.
        lower: float
            Lower boundary.
        equal: float
            Equal boundary.
        tol: float
            Tolerance for equal boundary; negative number for percent,
            positive number for absolute value.

        Note: the attributes 'lower' and 'upper' in the pyOpt.constraint
        object does not take effect. In the new Constraint object,
        the normalize() method will convert the value corresponding to
        a constraint type of (-INF, 0] which can then pass to pyOpt.
        """
```

```
class Variable(pyOptVariable):
    """Inherited from pyOpt.Variable class"""
    def __init__(self, name, **kwargs):
        """Initialize Constraint object.

        Parameters
        ----------
        type: string
            Variable Type ('c'-continuous, 'i'-integer, 'd'-discrete),
            *Default* = 'c'
        value: int/float
            Variable Value, *Default* = 0.0
        upper: int/float
            Upper boundary.
        lower: int/float
            Lower boundary.
        """
        

class CoVariable(object):
    """Optimization CoVariable class."""
    def __init__(self, name, var_dp, **kwargs):
        """Initialize CoVariable object

        The value of the variable is calculated by:
        covar = [slope].*[value of var_dp] + intercept

        Parameters
        ----------
        name: string
            Name of the co-variable.
        var_dep: string/list
            Name(s) of the dependent variable(s).
        slope: int/float/list
            Coefficient(s) in calculation.
        intercept: int/float
            Coefficient in calculation.
        """
        
class StaticVariable(object):
    """Optimization StaticVariable class

    Static variable is useful when running concatenate simulations.
    """

    def __init__(self, name, value=None):
        """Initialize StaticVariable object

        Parameters
        ----------
        name: string
            Name of the static variable.
        value: int/float
            Value of the static variable. The value will be ignored
            if a variable with the same name is found in the variable
            set or co-variable set of the solution.
        """
```

```
def solve(self, run_code=None):
    """Run the optimization and print the result.

    Parameters
    ----------
    run_code: string
        String that can run the code in the shell.
    """
```